# Chapter 08: 공급망 물류 에이전트

이 노트북에서는 공급망 및 물류 관리를 위한 AI 에이전트를 구축합니다.

## 주요 내용
- 다양한 물류 도구 정의
- 복잡한 시스템 프롬프트 설계
- 도구 호출 및 결과 처리

⚠️ **참고**: 원본 코드의 Traceloop/Loki 관측성 기능은 Colab 호환을 위해 제거되었습니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch08_supply_chain_agent.ipynb)


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv


In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"


## 1. 물류 도구 정의


In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict, Optional
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph

# Loki 대체 함수 (Colab용)
def log_to_loki(label, message):
    print(f"[LOG] {label}: {message}")

@tool
def manage_inventory(sku: Optional[str] = None, **kwargs) -> str:
    """재고 수준, 재고 보충, 감사 및 최적화 전략을 관리합니다."""
    print(f"[도구] manage_inventory(sku={sku})")
    return "inventory_management_initiated"

@tool
def track_shipments(origin: Optional[str] = None, **kwargs) -> str:
    """배송 상태, 지연 사항을 추적하고 배송 물류를 조정합니다."""
    print(f"[도구] track_shipments(origin={origin})")
    return "shipment_tracking_updated"

@tool
def forecast_demand(season: Optional[str] = None, **kwargs) -> str:
    """수요 패턴, 계절적 추세를 분석하고 예측 모델을 생성합니다."""
    print(f"[도구] forecast_demand(season={season})")
    return "demand_forecast_generated"

@tool
def send_logistics_response(operation_id: Optional[str] = None, message: Optional[str] = None) -> str:
    """이해관계자에게 물류 업데이트 또는 권장 사항을 전송합니다."""
    print(f"[도구] send_logistics_response → {message}")
    return "logistics_response_sent"

TOOLS = [manage_inventory, track_shipments, forecast_demand, send_logistics_response]
print("✅ 물류 도구 정의 완료:", [t.name for t in TOOLS])


## 2. 공급망 에이전트 구축


In [ ]:
# LLM 초기화 및 도구 바인딩
llm = init_chat_model(model="gpt-5-mini", verbose=True).bind_tools(TOOLS)

class AgentState(TypedDict):
    operation: Optional[dict]
    messages: Annotated[Sequence[BaseMessage], operator.add]

def call_model(state: AgentState):
    history = state["messages"]
    operation = state.get("operation", {})
    if not operation:
        operation = {"operation_id": "UNKNOWN", "type": "general"}
    
    operation_json = json.dumps(operation, ensure_ascii=False)
    system_prompt = f"""당신은 숙련된 공급망 및 물류 관리 전문가입니다.

전문 분야:
- 재고 관리 및 수요 예측
- 운송 및 배송 최적화
- 품질 관리 및 비용 최적화

공급망 운영을 관리할 때:
1) 물류 과제를 분석합니다
2) 적절한 공급망 관리 도구를 호출합니다
3) send_logistics_response로 권장 사항을 제공합니다

작업: {operation_json}"""

    full = [SystemMessage(content=system_prompt)] + history
    first = llm.invoke(full)
    messages = [first]

    if getattr(first, "tool_calls", None):
        for tc in first.tool_calls:
            fn = next(t for t in TOOLS if t.name == tc['name'])
            out = fn.invoke(tc["args"])
            messages.append(ToolMessage(content=str(out), tool_call_id=tc["id"]))
        second = llm.invoke(full + messages)
        messages.append(second)

    return {"messages": messages}

def construct_graph():
    g = StateGraph(AgentState)
    g.add_node("assistant", call_model)
    g.set_entry_point("assistant")
    return g.compile()

graph = construct_graph()
print("✅ 공급망 에이전트 그래프 구성 완료")


In [ ]:
# 테스트 실행
example = {
    "operation_id": "OP-12345", 
    "type": "inventory_management", 
    "priority": "high", 
    "location": "Warehouse A"
}
convo = [HumanMessage(content="SKU-12345 재고가 심각하게 부족합니다. 현재 재고는 50개이지만 미처리 주문이 200개입니다. 재주문 전략은 무엇입니까?")]

result = graph.invoke({"operation": example, "messages": convo})

print("\n" + "=" * 60)
print("🏭 공급망 에이전트 응답")
print("=" * 60)
for m in result["messages"]:
    print(f"\n{m.type}: {m.content}")
